# Initial Setups

## Set MSCTD Configurations

In [1]:
!pip install --upgrade --no-cache-dir gdown
!pip3 install --upgrade gdown
!pip install -U --no-cache-dir gdown --pre
!pip install --upgrade --no-cache-dir gdown
# 
!git clone https://github.com/XL2248/MSCTD

!cp MSCTD/MSCTD_data/ende/english_*.txt .
!cp MSCTD/MSCTD_data/ende/image_index_*.txt .
!cp MSCTD/MSCTD_data/ende/sentiment*.txt .

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'MSCTD'...
remote: Enumerating objects: 1217, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1217 (delta 13), reused 7 (delta 3), pack-reused 1190
Receiving objects: 100% (1217/1217), 102.24 MiB | 14.09 MiB/s, done.
Resolving deltas: 100% (617/617), done.
Updating files: 100% (934/934), done.
Mounted at /content/drive


In [2]:
!pip install einops
!pip install pytorch_lightning
!pip install wandb
!pip install ml_collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 52.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 22.5 MB/s eta 0:00:00
  Created wheel fo

## Import Libraries

In [3]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
from tqdm import tqdm
from time import localtime
import gc

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from IPython.display import clear_output 
from PIL import Image, ImageOps, ImageEnhance

import pytorch_lightning as pl
import torchvision.datasets as dset
import torchvision.transforms as T
from matplotlib import image as mpimg
from PIL import Image
from PIL import Image, ImageOps, ImageEnhance
from einops import parse_shape, rearrange
from opt_einsum import contract
import math
import functools
import pickle, marshal
from einops import rearrange
from numpy import asarray
import random
import torchvision.transforms as transforms

## Device Configuration

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Create Data Loader Class

In [5]:
def save_function(function, directory):
    code_string = marshal.dumps(function.__code__)
    with open(directory, 'wb') as file:
        pickle.dump(code_string, file)
    file.close()

In [6]:
def ToNumpy():
    return pd.DataFrame.to_numpy
def ToTensor():
    return transforms.ToTensor

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, path_label, path_sentiment, path_dialoges, drive_id, mode,transform, image_size, batch_size=None):
    'Initialization'
    self.path_label = path_label
    self.path_sentiment = path_sentiment
    self.path_dialoges = path_dialoges
    self.drive_id = drive_id
    self.mode = mode
    self.image_size = image_size
    self.batch_size = batch_size
    self.ds = self.load_images()
    self.labels = np. asarray(self.read_txt(path_label)[:-1])
    self.indexes = np. asarray(self.conversations_index()[:-1])
    self.dialoges = np. asarray(self.read_txt(path_dialoges)[:-1])
    self.transform = transform
    self.convertType()

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.labels)

  def __getitem__(self, index):
    'Generates one sample of data'
    image_index = self.get_image(index)
    dialoge_index = self.dialoges[index]
    label_index = self.labels[index]
    return image_index, dialoge_index, label_index
  
  def read_txt(self, path_text):
    'Reads text and turn them into a list'
    my_file = open(path_text, "r")
    data = my_file.read()
    data_into_list = data.split("\n")
    return data_into_list

  def load_images(self):
    'download images'
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_id = self.drive_id
    file_path = './DLproject'
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    path_zip = '/content/' + self.mode + '.zip'
    !gdown --id "$file_id"
    !unzip "$path_zip" -d '/content/DLproject/'

    img_height, img_width = self.image_size
    dir_files = '/content/DLproject'
    ds = tf.keras.utils.image_dataset_from_directory(
        dir_files, image_size=(img_height, img_width), batch_size=self.batch_size
    )
    return ds

  def get_image(self, index):
    file_name = '/content/DLproject/'+ self.mode + '//'+ str(index) + '.jpg'
    image = cv2.imread(file_name)
    image= np.asarray(image)
    if(self.transform == transforms.ToTensor):
        trans = transforms.Compose([transforms.ToTensor()])
        image = trans(image)

    return image

  def conversations_index(self):
    'map dialoge index to an image'
    my_file = open(self.path_sentiment, "r")
    data = my_file.read()
    conversations = data.split("\n")
    indexes = []
    for i in range(len(conversations)):
      conversation_i = conversations[i]
      conversation_i = conversation_i[1: -1]
      conversation_i = conversation_i.split(",")
      dialoges_per_conversation = len(conversation_i)
      for j in range(dialoges_per_conversation):
        indexes.append(i)
    return indexes

  def convertType(self):
      if(self.transform == transforms.ToTensor):
          self.dialoges = tf.convert_to_tensor(self.dialoges)
          self.indexes = tf.convert_to_tensor(self.indexes)
          self.labels = tf.convert_to_tensor(self.indexes)

## Import Dataset and Create Dataset Object

In [8]:
classes = ['Neutral', 'Negative', 'Positive']

In [11]:
id = '1EYjMT2_0EpmZQSbWtsaQOMkP_9Zxt8Ll'
data_loader_train = Dataset('sentiment_train.txt', 'image_index_train.txt', 'english_train.txt', id, 'train_ende',transform=ToNumpy(), image_size=(554,1280), batch_size=64)
clear_output()

In [ ]:
# id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
# data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToTensor(), image_size=(554,1280), batch_size=64)
# clear_output()

In [12]:
id = '1_kjhrE_503e1ahPFycIi9sev_7nOq6T3'
data_loader_test = Dataset('sentiment_test.txt', 'image_index_test.txt', 'english_test.txt', id, 'test',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

In [13]:
id = '1SlsoJ0X4C-IK6CQQ7L93Od4MrK6TRPv9'
data_loader_val = Dataset('sentiment_dev.txt', 'image_index_dev.txt', 'english_dev.txt', id, 'dev',transform=ToNumpy(),image_size=(554,1280), batch_size=64)
clear_output()

# Part A-2

### Installing 

In [20]:
!pip install torchmultimodal-nightly
#!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.0 MB/s eta 0:00:00
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=1f8b626f41eeee35b4b60028f0d5646b16f5f998071a6cd466e7c4f844b0920c
  Stored in directory: /root/.cache/pip/wheels/89/3e/24/0f349c0b2eeb6965903035f3b00dbb5c9bea437b4a2f18d82c
Successfully built iopath
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.7 MB/s eta 0:00

### Processing Data

In [14]:
import re
def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """This utility function sanitizes a string by:
    - removing links
    - removing special characters
    - removing numbers
    - removing stopwords
    - transforming in lowercase
    - removing excessive whitespaces
    Args:
        text (str): the input text you want to clean
        remove_stopwords (bool): whether or not to remove stopwords
    Returns:
        str: the cleaned text
    """

    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. tokenize
        tokens = nltk.word_tokenize(text)
        # 2. check if stopword
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        # 3. join back together
        text = " ".join(tokens)
    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()
    return text

In [15]:
def string_label(x):
    if x == '2':
        return 'Positive' 
    elif x == '1':
        return 'Negative' 
    else:
        return 'Neutral'

In [16]:
import os
import glob
import cv2
import h5py
import numpy as np
import pandas as pd
import pandas as pd

def images_to_hdf5(data_loader: Dataset, frame_size = (128, 128),skip=1,
                   hdf5_path='./hdf5_file/file.hdf5'):
    data=[]
    df=pd.DataFrame(data,columns=["image","dialoges","target"])
    N = data_loader.__len__()
    #hdf5_file = h5py.File(hdf5_path, mode='w')
    #hdf5_file.create_dataset("image", (N, 224, 224,3), np.float32)
    #hdf5_file.create_dataset("answers", (N,), h5py.special_dtype(vlen=str))
    #hdf5_file.create_dataset("question", (N,), h5py.special_dtype(vlen=str))

    for i in tqdm(range(0, N, skip)):
        frame, dialog , label = data_loader.__getitem__(i)       
        img = cv2.resize(frame, (224, 224)).astype('float32')  # shape (224, 224, 3)
        #img = np.rollaxis(img, 2)[None]  # shape (1, 3, 224, 224)
        #hdf5_file["image"][i, ...] = img
        #hdf5_file["answers"][i, ...] = string_label(label)
        #hdf5_file["question"][i, ...] = dialog
        dialog = preprocess_text(dialog, remove_stopwords=False)
        new_row = {"image":img , "dialoges" :dialog,"target" :int(label)}
        df = df.append(new_row, ignore_index=True)
        #df.loc[str(i)] = [img, dialog,string_label(label)]


    #hdf5_file.close()
    return df

In [36]:
df_test = images_to_hdf5(data_loader_test, frame_size = (128, 128),skip=2,hdf5_path='testDataset.hdf5')
df_train = images_to_hdf5(data_loader_train, frame_size = (128, 128),skip=2,hdf5_path='trainDataset.hdf5')
df_val = images_to_hdf5(data_loader_val, frame_size = (128, 128),skip=2,hdf5_path='valDataset.hdf5')

  0%|          | 0/2534 [00:00<?, ?it/s]

  0%|          | 0/10120 [00:00<?, ?it/s]

  0%|          | 0/2532 [00:00<?, ?it/s]

In [37]:
df_test.head()

,image,dialoges,target
0,"[[[252.0, 252.0, 252.0], [252.0, 252.0, 252.0]...",with this asshole,1
1,"[[[19.0, 15.0, 14.0], [19.0, 15.0, 10.0], [15....",i m not happy about it either okay at all,1
2,"[[[22.0, 17.0, 16.0], [21.0, 16.0, 15.0], [22....",you re on admin leave until they make a determ...,0
3,"[[[16.0, 33.0, 24.0], [16.0, 33.0, 24.0], [16....",what do you mean,0
4,"[[[16.0, 33.0, 24.0], [17.0, 32.0, 24.0], [18....",rather than guard every project,0


In [19]:
class Dataset2(torch.utils.data.Dataset):

    def __init__(self, tokenizer, images ,dialoge_texts , targets=None, labels=None, max_len=10):
        self.dialoge_texts = dialoge_texts
        self.targets = targets
        self.images = images
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)
        self.label_map = {label: i for i, label in enumerate(labels)} if isinstance(labels, list) else {}
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.image_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize([224,224])])
    
    def __len__(self):
        return len(self.dialoge_texts)

    def __getitem__(self, item):
        dialoge_text = self.dialoge_texts[item]
        image =  self.image_transform(self.images[item])

        if self.has_target:
            target = self.label_map.get(self.targets[item], self.targets[item])
        encoding = self.tokenizer.encode_plus(
            dialoge_text, add_special_tokens=True, truncation=True, max_length=self.max_len, return_token_type_ids=True,
            padding='max_length', return_attention_mask=True, return_tensors='pt')
        
        inputs = { 'image': image ,'dialoge_text': dialoge_text, 'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(), 'token_type_ids': encoding['token_type_ids'].flatten(),}

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
            
        return image, dialoge_text , torch.tensor(target, dtype=torch.long)

In [38]:
batch_size = 16
from transformers import BertConfig, BertTokenizer
label_map = ['neutral', 'negative', 'positive']
label2id = {label: i for i, label in enumerate(label_map)}
id2label = {v: k for k, v in label2id.items()}

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = Dataset2(tokenizer,  df_train['image'] ,df_train['dialoges'].to_numpy(), df_train['target'].to_numpy(), label_map, 10)
valid_dataset = Dataset2(tokenizer, df_val['image'], df_val['dialoges'].to_numpy() , df_val['target'].to_numpy(), label_map, 10)
test_dataset = Dataset2(tokenizer,df_test['image'], df_test['dialoges'].to_numpy(), df_test['target'].to_numpy(), label_map, 10)

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [39]:
for batch in train_data_loader:
    imgs, txts, labels = batch
    print(imgs.shape)
    print(len(txts))
    print(labels)
    print(labels.shape)
    print(torch.unique(labels).shape)
    break

torch.Size([16, 3, 224, 224])
16
tensor([2, 1, 2, 2, 2, 2, 1, 1, 0, 1, 2, 1, 0, 0, 2, 0])
torch.Size([16])
torch.Size([3])


In [40]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-nzpy6vqg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-nzpy6vqg
  Resolved https://github.com/openai/CLIP.git to commit 3702849800aa56e2223035bccd1c6ef91c704ca8
  Preparing metadata (setup.py) ... done


### Training & Testing

In [41]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import torchtext
import nltk, re, string, collections
from nltk.util import ngrams
import collections
%matplotlib inline
BATCH_SIZE = 128
EPOCH = 5

In [42]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

#text = clip.tokenize(['a diagram', "a dog", "a cat", "ss"]).to(device)
#text.shape

In [43]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_data_loader)*EPOCH)

In [27]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(32, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 3),
            nn.Softmax(-1)
        )
        
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        #x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [44]:
logits_train = []
logits_val = [] 
target_train = []
target_val = []

In [45]:
best_te_loss = 1e5
best_ep = -1
bestmodel = model
for epoch in range(EPOCH):
    print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_data_loader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, target = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
        #print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        #print(logits_per_image.shape ,logits_per_text.shape )
        ground_truth = torch.arange(16).to(device)
        # if(images.shape[0]!=16):
        #     break
        #print(loss_img(logits_per_image,ground_truth))
        try:
            if(torch.cat((logits_per_image, logits_per_text), -1).flatten().shape[0]==512):
                logits_train.append(torch.cat((logits_per_image, logits_per_text), -1).flatten())
                target_train.append(target)
            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            total_loss.backward()
            tr_loss += total_loss.item()
            if device == "cpu":
                optimizer.step()
                scheduler.step()
            else:
                convert_models_to_fp32(model)
                optimizer.step()
                scheduler.step()
                clip.model.convert_weights(model)
        except:
            print()
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step
    
    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(valid_data_loader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, target = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(16).to(device)
            #if(images.shape[0]!=16):
            #    break
            try:
                if(torch.cat((logits_per_image, logits_per_text), -1).flatten().shape[0]==512):
                    logits_val.append(torch.cat((logits_per_image, logits_per_text), -1).flatten())
                    target_val.append(target)
                total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
                te_loss += total_loss.item()
            except:
                test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step
        
    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
        torch.save(model.state_dict(), "best_model.pt")
        bestmodel = model
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model.pt")

running epoch 0, best test loss 100000.0 after epoch -1


  0%|          | 0/633 [00:00<?, ?it/s]

  0%|          | 0/159 [00:00<?, ?it/s]

epoch 0, tr_loss 2.7416999901263823, te_loss 2.6892565841194966
running epoch 1, best test loss 2.6892565841194966 after epoch 0


  0%|          | 0/633 [00:00<?, ?it/s]

...

  0%|          | 0/159 [00:00<?, ?it/s]

epoch 3, tr_loss 0.5065917004529522, te_loss 4.887812008647798
running epoch 4, best test loss 2.6892565841194966 after epoch 0


  0%|          | 0/633 [00:00<?, ?it/s]

  0%|          | 0/159 [00:00<?, ?it/s]

epoch 4, tr_loss 0.31606735060753605, te_loss 4.988674331761007


In [46]:
drive_path = F"/content/drive/MyDrive/transformer" 
torch.save(bestmodel, drive_path)

In [47]:
logits_test = []
target_test = []

In [49]:
test_loss  = 0
step = 0
with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_data_loader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, target = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = bestmodel(images, texts)
            ground_truth = torch.arange(16).to(device)
            if(images.shape[0]!=16):
                break
            if(torch.cat((logits_per_image, logits_per_text), -1).flatten().shape[0]==512):
                logits_test.append(torch.cat((logits_per_image, logits_per_text), -1).flatten())
                target_test.append(target)
            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            test_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        test_loss /= step
print(f" test_loss {test_loss}")

  0%|          | 0/159 [00:00<?, ?it/s]

 test_loss 5.199710102201258


In [50]:
logits_train = torch.stack(logits_train)
logits_val = torch.stack(logits_val)
target_train = torch.stack(target_train)
target_val = torch.stack(target_val)
logits_test = torch.stack(logits_test)
target_test = torch.stack(target_test)

In [51]:
train_list = []
for i in range(0,logits_train.shape[1],32):
    train_list.append(logits_train[:,i:32+i])
val_list = []
for i in range(0,logits_val.shape[1],32):
    val_list.append(logits_val[:,i:32+i])
test_list = []
for i in range(0,logits_test.shape[1],32):
    test_list.append(logits_test[:,i:32+i])

In [52]:
tain_list2 = torch.stack(train_list).flatten(end_dim=1).cpu().detach()

In [53]:
val_list2 = torch.stack(val_list).flatten(end_dim=1).cpu().detach()

In [54]:
test_list2 = torch.stack(test_list).flatten(end_dim=1).cpu().detach()

In [55]:
train_target2 = target_train.flatten().cpu().detach()

In [56]:
val_target2 = target_val.flatten().cpu().detach()

In [57]:
test_target2 = target_test.flatten().cpu().detach()

In [64]:
test_list2.shape

torch.Size([5056, 32])

In [61]:
test_target2.shape

torch.Size([5056])

In [65]:
batch_size = 64
train_set = list(map(lambda x, y: (x, y), tain_list2, train_target2))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_set = list(map(lambda x, y: (x, y), val_list2,  val_target2))
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_set = list(map(lambda x, y: (x, y), test_list2,  test_target2))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [66]:
model = MLP()
print(model)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=32, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.1, inplace=False)
    (5): Linear(in_features=512, out_features=256, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=256, out_features=3, bias=True)
    (9): Softmax(dim=-1)
  )
)


In [67]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()


In [68]:
mean_train_losses = []
mean_valid_losses = []
valid_acc_list = []
epochs = 15
acc = 0
best_model = model
for epoch in range(epochs):
    model.train()
    
    train_losses = []
    valid_losses = []
    for i, (images, labels) in enumerate(train_loader):
        if(labels.shape[0]!=64):
            break
        labels = labels.type(torch.LongTensor)
        images = images.type(torch.float32)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if (i * 128) % (128 * 100) == 0:
            print(f'{i * 128} / 50000')
            
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(val_loader):
            if(labels.shape[0]!=64):
                break
            labels = labels.type(torch.LongTensor)
            images = images.type(torch.float32)
            #print(labels.shape)
            #print(images.shape)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            
            valid_losses.append(loss.item())
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    
    accuracy = 100*correct/total
    valid_acc_list.append(accuracy)
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.2f}%'\
         .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))
    if(accuracy>acc):
        best_model = model
        acc = accuracy
print("BEST VAL ACC= ",acc)


0 / 50000
12800 / 50000
25600 / 50000
38400 / 50000
51200 / 50000
64000 / 50000
76800 / 50000
89600 / 50000
epoch : 1, train loss : 1.1641, valid loss : 1.1824, valid acc : 36.91%
0 / 50000
12800 / 50000
25600 / 50000
38400 / 50000
51200 / 50000
64000 / 50000
76800 / 50000
89600 / 50000
epoch : 2, train loss : 1.1651, valid loss : 1.1829, valid acc : 36.86%
0 / 50000
12800 / 50000
25600 / 50000
38400 / 50000
51200 / 50000
64000 / 50000
76800 / 50000
89600 / 50000
epoch : 3, train loss : 1.1651, valid loss : 1.1824, valid acc : 36.91%
0 / 50000
12800 / 50000
25600 / 50000
38400 / 50000
51200 / 50000
64000 / 50000
76800 / 50000
89600 / 50000
epoch : 4, train loss : 1.1651, valid loss : 1.1826, valid acc : 36.88%
0 / 50000
12800 / 50000
25600 / 50000
38400 / 50000
51200 / 50000
64000 / 50000
76800 / 50000
89600 / 50000
epoch : 5, train loss : 1.1651, valid loss : 1.1821, valid acc : 36.94%
0 / 50000
12800 / 50000
25600 / 50000
38400 / 50000
51200 / 50000
64000 / 50000
76800 / 50000
89600 

In [72]:
best_model.eval()
correct = 0
total = 0
test_losses = []
mean_test_losses = []
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        if(labels.shape[0]!=64):
            break
        labels = labels.type(torch.LongTensor)
        images = images.type(torch.float32)
            #print(labels.shape)
            #print(images.shape)
        outputs = best_model(images)
        loss = loss_fn(outputs, labels)
            
        test_losses.append(loss.item())
            
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
            
mean_test_losses.append(np.mean(test_losses))

    
accuracy = 100*correct/total

print(' test loss : {:.4f}, test acc : {:.2f}%'\
         .format( np.mean(test_losses),  accuracy))

 test loss : 1.1195, test acc : 43.20%


In [73]:
drive_path = F"/content/drive/MyDrive/MLP_classifier" 
torch.save(model, drive_path)


In [74]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

Sun Feb 12 10:24:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    51W / 400W |   4650MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [75]:
torch.cuda.empty_cache()
gc.collect()

0

### FLAVA

In [ ]:
from torch.optim import Adam, SGD
from transformers import get_linear_schedule_with_warmup
path = '/content/Emotion_Classification_Bert/pytorch_model.bin'
EPOCHS = 3
from torchmultimodal.models.flava.model import flava_model_for_classification
model = flava_model_for_classification(num_classes=3)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters())
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

criterion = nn.CrossEntropyLoss()

step = 0
minimum_loss = np.Inf


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs"):

    train_y, train_loss, step, minimum_loss = train_model(model, train_data_loader, criterion, optimizer, scheduler, step,
    True, minimum_loss, valid_data_loader, epoch, EPOCHS, path)
    
    train_score = score(train_y[0], train_y[1], average='weighted')
    y_val, loss_val = evaluate_model(model, valid_data_loader, criterion)
    valid_score = score(y_val[0], y_val[1], average='weighted')